In [ ]:
import pandahouse as ph

In [ ]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student',
                      'password':'dpo_python_2020'
                     }

In [ ]:

query = '''
SELECT
  COUNT(st_id) as qty_of_hard_students
FROM
  (
    SELECT
      st_id,
      SUM(correct) as ready_per_month
    FROM
      default.peas
    GROUP BY
      st_id
  ) as sq
WHERE
  ready_per_month >= 20
'''

st_qty = ph.read_clickhouse(query=query, connection=connection_default)
st_qty

In [ ]:
metrics_query = '''

WITH active_users AS (
  SELECT
    st_id
  FROM
    default.peas
  GROUP BY
    st_id
  HAVING
    SUM(correct) > 10
),
math_active_users AS (
  SELECT
    st_id
  FROM
    default.peas
    WHERE subject = 'Math'
  GROUP BY
    st_id
  HAVING
    SUM(correct) >= 2
)

SELECT
  test_grp AS Groups,
  SUM(default.final_project_check.money) / CountDistinct(default.studs.st_id) AS ARPU,
  SUM(default.final_project_check.money) FILTER (
    WHERE
      default.studs.st_id IN active_users
  ) / CountDistinct(default.studs.st_id) FILTER (
    WHERE
      default.studs.st_id IN active_users
  ) AS ARPAU,
  CountDistinct(default.studs.st_id) FILTER (
    WHERE
      default.final_project_check.money != 0
  ) / CountDistinct(default.studs.st_id) AS CR,
  CountDistinct(default.studs.st_id) FILTER (
    WHERE
      default.final_project_check.money != 0
      AND default.studs.st_id IN active_users
  ) / CountDistinct(default.studs.st_id) FILTER (
    WHERE
      default.studs.st_id IN active_users
  ) AS CR_active_users,

   CountDistinct(default.studs.st_id) FILTER (
    WHERE
      default.final_project_check.money != 0
      AND default.studs.st_id IN math_active_users
      AND default.final_project_check.subject = 'Math'
  ) / CountDistinct(default.studs.st_id) FILTER (
    WHERE
      default.studs.st_id IN math_active_users
  ) AS CR_math_active_users
  FROM
  default.studs
  LEFT JOIN default.final_project_check on default.final_project_check.st_id = default.studs.st_id
GROUP BY
  test_grp
'''

metrics = round(ph.read_clickhouse(query=metrics_query, connection=connection_default),2)
metrics